In [1]:
import os
import sys
import yaml
import pickle

from stable_baselines3 import DQN
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import EvalCallback

# Llamar al entorno
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from src.environment_dqn import RetailEnvDQN

# cargar stores.pkl procesado
STORES_PATH = "../data/stores.pkl"
with open(STORES_PATH, "rb") as f:
    stores = pickle.load(f)

print(f"{len(stores)} tiendas cargadas desde stores.pkl")

# directorio con los parámetros de configuración
CONFIG_FILE = "../configs/production.yaml"     # ultra_fast o production(mejor calidad, pero más lento)
with open(CONFIG_FILE, "r") as f:
    cfg = yaml.safe_load(f)


# Se crea una carpeta para experimentos
experiment_dir = f"../results/models/dqn/"
os.makedirs(experiment_dir, exist_ok=True)

# 1. Split por tiendas (no por filas)
store_ids = list(stores.keys())
store_ids.sort()

split_idx = int(len(store_ids) * 0.8)
train_stores = {sid: stores[sid] for sid in store_ids[:split_idx]}
eval_stores  = {sid: stores[sid] for sid in store_ids[split_idx:]}

def make_train_env(rank):
    """Cada env escribe su propio monitor.csv para evitar colisiones."""
    def _init():
        path = os.path.join(experiment_dir, f"monitor_train_{rank}.csv")
        env = RetailEnvDQN(train_stores)
        env.reset(seed=rank)
        return Monitor(env, filename=path)
    return _init


def make_eval_env(rank):
    """Eval env separado, sin normalización de recompensa."""
    def _init():
        path = os.path.join(experiment_dir, f"monitor_eval_{rank}.csv")
        env = RetailEnvDQN(eval_stores)
        env.disable_random_reset = True
        env.fixed_start = 0
        env.reset(seed=1000 + rank)
        return Monitor(env, filename=path)
    return _init


# Entrenamiento
N_ENVS = cfg["n_envs"]

# Inicializar entornos
train_env = DummyVecEnv([make_train_env(i) for i in range(N_ENVS)])
eval_env = DummyVecEnv([make_eval_env(0)])


# Configuración del DQN con los parámetros del archivo de configuración
policy_kwargs = dict(net_arch=cfg["policy_net"]["layers"])

model = DQN(
    "MlpPolicy",
    train_env,
    verbose=1,
    learning_rate=cfg["dqn"]["learning_rate"],
    gamma=cfg["dqn"]["gamma"],
    buffer_size=cfg["dqn"]["buffer_size"],
    learning_starts=cfg["dqn"]["learning_starts"],
    batch_size=cfg["dqn"]["batch_size"],
    train_freq=cfg["dqn"]["train_freq"],
    gradient_steps=cfg["dqn"]["gradient_steps"],
    n_steps=cfg["dqn"]["n_steps"],
    target_update_interval=cfg["dqn"]["target_update_interval"],
    exploration_fraction=cfg["exploration"]["fraction"],
    exploration_initial_eps=cfg["exploration"]["initial_eps"],
    exploration_final_eps=cfg["exploration"]["final_eps"],
    policy_kwargs=policy_kwargs,
    tensorboard_log=os.path.join(experiment_dir, "tensorboard")
)

# Capturando el mejor modelo 
callback = EvalCallback(
    eval_env,
    best_model_save_path=os.path.join(experiment_dir, "best_model"),
    log_path=os.path.join(experiment_dir, "eval_logs"),
    eval_freq=10_000,
    deterministic=True,
    render=False,
)


TOTAL_STEPS = cfg["total_steps"]

print(f"Entrenando por {TOTAL_STEPS:,} pasos\n")

model.learn(
    total_timesteps=TOTAL_STEPS,
    callback=callback
)

print("\nEntrenamiento finalizado.")

# Guardando el modelo final 
model_final_dir = os.path.join(experiment_dir, "model_final")
os.makedirs(model_final_dir, exist_ok=True)

model.save(os.path.join(model_final_dir, "dqn_final.zip"))

print("Modelo final guardado:")
print(f"   → {model_final_dir}/dqn_final.zip")
print("\nResultados completos en:")
print(f"   {experiment_dir}")


5 tiendas cargadas desde stores.pkl
Using cpu device
Entrenando por 1,000,000 pasos

Logging to ../results/models/dqn/tensorboard/DQN_25
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 90       |
|    ep_rew_mean      | 53.5     |
|    exploration_rate | 0.999    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 5324     |
|    time_elapsed     | 0        |
|    total_timesteps  | 360      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 90       |
|    ep_rew_mean      | 54.7     |
|    exploration_rate | 0.998    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 5804     |
|    time_elapsed     | 0        |
|    total_timesteps  | 720      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      |